In [34]:
import time
import pandas as pd
import dask.dataframe as dd
import numpy as np

from epiweeks import Week

In [3]:
vac_url = '../RawData/TB_VACUNACION_COVID19.csv'
fal_url = '../RawData/fallecidos_covid.csv'

In [ ]:
start = time.time()
ddf = dd.read_csv(vac_url, sep = "," )
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
ddf.head()

In [ ]:
ddf = ddf.assign(fecha_vacunacion = dd.to_datetime(ddf["fecha_vacunacion"], format = "%Y%m%d", errors="coerce"))
ddf.head()

In [ ]:
ddf['isoformat'] = ddf['fecha_vacunacion'].apply(lambda date : Week.fromdate(date).isoformat(), meta=('isoformat', 'object'))
ddf.head()

In [ ]:
ddf[['epi_week4','epi_week5']] = ddf['isoformat'].str.split("W", 1, expand=True)
ddf.head()

In [ ]:
%reset 

In [ ]:
#ddf.loc[ddf['dosis'] == 4].compute()
#ddf.loc[ddf['dosis'] == 5].compute()
#ddf.groupby("id_vacuna").edad.mean().compute()
#ddf.partitions[4].compute()
#type(ddf.partitions[4].compute())
#print("Número de dosis aplicadas:"), 
#ddf.dosis.value_counts().compute()
#ddf.map_partitions(len).compute()

#def variable_fecha(ddf, date_name):
#    return ddf.assign(fecha_vacunacion = dd.to_datetime(ddf[date_name], format = "%Y%m%d", errors="coerce"))

#ddf = ddf.map_partitions(variable_fecha, "fecha_vacunacion")
#ddf.head(10)

##test = dd.to_datetime(ddf["fecha_vacunacion"], format = "%Y%m%d").compute().to_frame()
#test

In [61]:
import pandas as pd
import dask.dataframe as dd

df = pd.DataFrame({'edad': [10, 20, 30, 40, 50],
                   'y': [1., 2., 3., 4., 5.]})
ddf = dd.from_pandas(df, npartitions=2)


def edad_cat(df, col):

    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]

    return np.select(conditions, choices, default=np.nan).astype('int8')


ddf['z'] = ddf.map_partitions(edad_cat, 'edad')


ddf.partitions[1].compute().head()

,edad,y,z
3,40,4.0,3
4,50,5.0,4


In [37]:
def variable_edad(df):

    conditions = [(df.edad< 18),
                  (df.edad>=18) & (df.edad<30),
                  (df.edad>=30) & (df.edad<40),
                  (df.edad>=40) & (df.edad<50),
                  (df.edad>=50) & (df.edad<60),
                  (df.edad>=60) & (df.edad<70),
                  (df.edad>=70) & (df.edad<80),
                  (df.edad>=80)]
    choices = [0,1,2,3,4,5,6,7]
    df = np.select(conditions, choices, default=np.nan)
    df = df.apply(np.int8)
    return df
ddf['new'] = ddf.map_partitions(variable_edad, meta=df.edad)


ddf.partitions[1].compute().head()

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

### ===========================================================================================================

In [54]:
df_fal = dd.read_csv(fal_url, sep = ';', dtype={'UBIGEO': 'object', 'id_persona': 'float64'},
                     usecols = ['FECHA_FALLECIMIENTO'])
df_fal = df_fal.assign(FECHA_FALLECIMIENTO = dd.to_datetime(df_fal["FECHA_FALLECIMIENTO"], format = "%Y%m%d", errors="coerce"))
df_fal

NameError: name 'fal_url' is not defined

In [ ]:
df_fal['isoformat'] = df_fal['FECHA_FALLECIMIENTO'].apply(lambda date : Week.fromdate(date).isoformat(), meta=('object'))
df_fal[['epi_week4','epi_week5']] = df_fal['isoformat'].str.split("W", 1, expand=True)
df_fal.head()

,FECHA_FALLECIMIENTO,isoformat,epi_week4,epi_week5
0,2021-06-11,2021W23,2021,23
1,2021-03-17,2021W11,2021,11
2,2021-06-02,2021W22,2021,22
3,2021-07-03,2021W26,2021,26
4,2021-05-06,2021W18,2021,18


In [ ]:
df_fal['epi_week1'] = df_fal['FECHA_FALLECIMIENTO'].apply(lambda date : Week.fromdate(date).cdcformat(), meta=('FECHA_FALLECIMIENTO', 'object'))
df_fal.head()

In [ ]:
df_fal['epi_week2'] = df_fal['FECHA_FALLECIMIENTO'].apply(lambda date : Week.fromdate(date).weektuple(), meta=('FECHA_FALLECIMIENTO', 'object'))
df_fal.head()

In [ ]:
df_fal['epi_week1'] = df_fal['epi_week1'].apply(np.uint32)
df_fal['epi_week4'] = df_fal['epi_week4'].apply(np.uint16)
df_fal['epi_week5'] = df_fal['epi_week5'].apply(np.uint8)
df_fal.head()

In [ ]:
df_fal.memory_usage()